In [1]:
import os, time, copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from PIL import Image

from utils import *
from dataset import *
from metrics import *
from FCN.fcn32_google import *

In [2]:
set_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"] = '3'


In [3]:
# Load data
train_data = VOC(mode = 'train', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
train_loader = data.DataLoader(train_data, batch_size=1, num_workers = 0, shuffle=True)

val_data = VOC(mode = 'val', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
val_loader = data.DataLoader(val_data, batch_size=1, num_workers = 0, shuffle=True)

In [4]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0

    loss_sum = torch.zeros(1)
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    optimizer.zero_grad()
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
            
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc += pixel_accuracy(out_mask, target_mask)/20
        macc += mean_accuracy(out_mask, target_mask)/20
        mIU += mean_IU(out_mask, target_mask)/20
        fIU += frequency_weighted_IU(out_mask, target_mask)/20
        
        #calculate loss
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)/20
        loss.backward()
        loss_sum += loss
        
        
        # update gradient every 20 image
        if i%20 == 19 or i == 1111:
            optimizer.step()
            print("Batch %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i//20, loss_sum, pacc, macc, mIU, fIU))
            
            running_loss += loss_sum.item() * 20
            r_pacc += pacc * 20
            r_macc += macc * 20
            r_mIU += mIU * 20
            r_fIU += fIU * 20
            
            # batch initialize
            loss_sum, pacc, macc, mIU, fIU = 0, 0, 0, 0, 0
            optimizer.zero_grad()
            
            # show output mask
#             img = decode_segmap(out_mask)
#             plt.imshow(img); plt.axis('off'); plt.show()
        
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1112 for m in epoch_metric]
    
    print('Train:\t%s' %epoch_metric)
    return epoch_metric

In [5]:
def validate(model, device, val_loader, criterion, mode = 'val'):
    # switch to evaluate mode
    model.eval()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc = pixel_accuracy(out_mask, target_mask)
        macc = mean_accuracy(out_mask, target_mask)
        mIU = mean_IU(out_mask, target_mask)
        fIU = frequency_weighted_IU(out_mask, target_mask)
        
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)
        
        if i % 100 == 99:
            print("Val %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i, loss, pacc, macc, mIU, fIU))
        # statistics
        running_loss += loss.item()
        r_pacc += pacc
        r_macc += macc
        r_mIU += mIU
        r_fIU += fIU

    #epoch_loss = running_loss / 1111
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1111 for m in epoch_metric]
    
    if mode == 'val':
        print('Val metric:\t%s' %epoch_metric)
    else:
        print('Test metric:\t%s' %epoch_metric)
    
    return epoch_metric  #, copy.deepcopy(model.state_dict())

In [6]:
def train_model(model, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=50):
    
    criterion = nn.CrossEntropyLoss(ignore_index = 255)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5**(-4))

    if device == 'cuda':
        model.to(device)
        cudnn.benchmark = True
        criterion = criterion.cuda()
    
    since = time.time()
    M = {'train' : [], 'val' : []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        train_metric = train(model, device, train_loader, criterion, optimizer)
        M['train'].append(train_metric)
        
        # evaluate on validation set
        val_metric = validate(model, device, val_loader, criterion)
        M['val'].append(val_metric)
        
        if val_metric[1] > best_acc:
            best_acc = val_metric[1]
            best_model_wts = copy.deepcopy(model.state_dict())
        
        if epoch % 20 == 19:
            SaveDict(model.state_dict(), dict_name+'_epoch'+str(epoch+60))
        print()
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    print(model.load_state_dict(best_model_wts))
    return best_model_wts, M

In [ ]:
LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32_google = FCN32_google()
fcn32_google_best, M = train_model(fcn32_google, device, train_loader, val_loader, lr = 5**(-5), dict_name = 'fcn32_google', num_epochs=200)

SaveDict(fcn32_google_best, 'fcn32_google_best')
SaveDict(M, 'fcn32_google_metric')

Epoch 0/199
----------
Batch 0)	3.030	0.001	0.017	0.000	0.000
Batch 1)	2.934	0.785	0.448	0.342	0.596
Batch 2)	2.812	0.668	0.392	0.272	0.477
Batch 3)	2.551	0.751	0.429	0.308	0.556
Batch 4)	2.291	0.758	0.406	0.305	0.574
Batch 5)	2.161	0.708	0.421	0.296	0.508
Batch 6)	2.040	0.686	0.446	0.304	0.488
Batch 7)	1.956	0.674	0.403	0.261	0.473
Batch 8)	1.863	0.697	0.431	0.281	0.506
Batch 9)	1.615	0.758	0.425	0.308	0.545
Batch 10)	1.538	0.778	0.421	0.321	0.587
Batch 11)	1.923	0.679	0.446	0.295	0.461
Batch 12)	2.113	0.640	0.393	0.242	0.426
Batch 13)	1.354	0.811	0.429	0.336	0.633
Batch 14)	1.582	0.777	0.442	0.326	0.598
Batch 15)	1.426	0.790	0.446	0.341	0.619
Batch 16)	1.725	0.741	0.417	0.302	0.569
Batch 17)	1.380	0.799	0.450	0.344	0.602
Batch 18)	1.758	0.734	0.425	0.301	0.538
Batch 19)	1.502	0.779	0.483	0.368	0.603
Batch 20)	1.665	0.746	0.433	0.320	0.551
Batch 21)	1.617	0.749	0.438	0.324	0.559
Batch 22)	1.706	0.716	0.427	0.294	0.511
Batch 23)	1.306	0.801	0.417	0.324	0.617
Batch 24)	1.572	0.723	0.458

Val 199)	1.271	0.669	0.500	0.325	0.421
Val 299)	1.629	0.510	0.500	0.247	0.244
Val 399)	0.756	0.952	0.250	0.227	0.822
Val 499)	1.288	0.660	0.332	0.188	0.312
Val 599)	0.723	0.816	0.250	0.187	0.561
Val 699)	1.067	0.736	0.500	0.354	0.502
Val 799)	0.288	0.967	0.333	0.310	0.866
Val 899)	0.799	0.836	0.500	0.395	0.626
Val 999)	1.395	0.726	0.500	0.355	0.503
Val 1099)	0.550	0.903	0.500	0.440	0.773
Val metric:	[1.1435610059075372, 0.7437298421625542, 0.4333742981309536, 0.31284914260453417, 0.5477057968619642]

Epoch 3/199
----------
Batch 0)	1.171	0.734	0.417	0.309	0.550
Batch 1)	1.168	0.749	0.400	0.292	0.535
Batch 2)	1.309	0.701	0.413	0.280	0.529
Batch 3)	0.986	0.796	0.437	0.338	0.600
Batch 4)	1.129	0.752	0.452	0.335	0.556
Batch 5)	1.208	0.729	0.425	0.295	0.533
Batch 6)	0.871	0.826	0.449	0.360	0.647
Batch 7)	1.064	0.762	0.446	0.332	0.568
Batch 8)	1.303	0.671	0.409	0.267	0.469
Batch 9)	1.329	0.720	0.419	0.302	0.536
Batch 10)	1.445	0.681	0.401	0.281	0.487
Batch 11)	1.127	0.750	0.429	0.305	0.550
B

Batch 47)	1.219	0.698	0.426	0.280	0.487
Batch 48)	1.020	0.792	0.458	0.357	0.627
Batch 49)	1.045	0.749	0.470	0.335	0.542
Batch 50)	1.099	0.740	0.430	0.310	0.545
Batch 51)	1.003	0.773	0.442	0.331	0.594
Batch 52)	0.870	0.819	0.428	0.338	0.630
Batch 53)	1.021	0.787	0.455	0.348	0.612
Batch 54)	1.036	0.781	0.452	0.341	0.599
Batch 55)	0.602	0.466	0.243	0.187	0.356
Train:	[1.1151299781078914, 0.7445614367279281, 0.436163686902008, 0.3167327018857443, 0.5502500536483547]
Val 99)	1.137	0.751	0.333	0.234	0.493
Val 199)	0.558	0.892	0.333	0.272	0.667
Val 299)	0.636	0.893	0.500	0.433	0.748
Val 399)	0.765	0.913	0.500	0.452	0.817
Val 499)	0.149	0.993	0.500	0.494	0.976
Val 599)	2.264	0.514	0.500	0.242	0.233
Val 699)	1.004	0.812	0.500	0.390	0.608
Val 799)	1.246	0.736	0.500	0.356	0.507
Val 899)	0.802	0.822	0.500	0.401	0.643
Val 999)	0.964	0.850	0.250	0.201	0.647
Val 1099)	1.557	0.417	0.354	0.149	0.174
Val metric:	[1.0738853415345202, 0.744912376097369, 0.4353112453559614, 0.3158017154025023, 0.5501076351

Batch 35)	1.083	0.726	0.407	0.307	0.543
Batch 36)	1.065	0.734	0.434	0.305	0.512
Batch 37)	1.044	0.763	0.458	0.337	0.580
Batch 38)	0.946	0.767	0.469	0.336	0.536
Batch 39)	1.172	0.744	0.460	0.325	0.557
Batch 40)	0.836	0.799	0.483	0.381	0.634
Batch 41)	0.968	0.774	0.461	0.350	0.600
Batch 42)	0.998	0.755	0.459	0.347	0.564
Batch 43)	0.959	0.761	0.416	0.322	0.560
Batch 44)	1.367	0.661	0.414	0.282	0.468
Batch 45)	0.824	0.806	0.439	0.337	0.615
Batch 46)	1.129	0.704	0.418	0.281	0.475
Batch 47)	1.058	0.745	0.434	0.313	0.556
Batch 48)	1.176	0.725	0.437	0.310	0.530
Batch 49)	1.274	0.700	0.429	0.301	0.509
Batch 50)	0.816	0.826	0.478	0.364	0.611
Batch 51)	0.871	0.798	0.461	0.361	0.619
Batch 52)	0.994	0.777	0.423	0.323	0.590
Batch 53)	0.937	0.785	0.469	0.364	0.607
Batch 54)	1.101	0.737	0.437	0.318	0.543
Batch 55)	0.608	0.462	0.279	0.205	0.335
Train:	[1.0584526685930842, 0.7481715386041835, 0.4430144769563837, 0.3245563350373435, 0.5559982984536573]
Val 99)	0.779	0.718	0.568	0.421	0.507
Val 199)	0.448

Batch 22)	1.146	0.702	0.444	0.302	0.478
Batch 23)	1.099	0.717	0.440	0.307	0.506
Batch 24)	0.902	0.783	0.465	0.345	0.594
Batch 25)	1.122	0.694	0.469	0.320	0.496
Batch 26)	1.033	0.744	0.410	0.293	0.555
Batch 27)	0.765	0.820	0.470	0.372	0.629
Batch 28)	0.902	0.788	0.479	0.368	0.623
Batch 29)	1.011	0.722	0.459	0.342	0.535
Batch 30)	0.950	0.770	0.458	0.348	0.580
Batch 31)	1.128	0.720	0.455	0.320	0.524
Batch 32)	1.286	0.668	0.470	0.297	0.441
Batch 33)	0.822	0.804	0.464	0.364	0.627
Batch 34)	1.136	0.738	0.474	0.350	0.539
Batch 35)	0.756	0.834	0.451	0.364	0.664
Batch 36)	0.975	0.778	0.453	0.341	0.578
Batch 37)	1.402	0.651	0.438	0.285	0.454
Batch 38)	0.836	0.797	0.470	0.367	0.606
Batch 39)	0.952	0.781	0.405	0.315	0.599
Batch 40)	0.912	0.785	0.477	0.364	0.564
Batch 41)	1.326	0.687	0.453	0.297	0.517
Batch 42)	1.113	0.705	0.464	0.328	0.531
Batch 43)	1.028	0.730	0.448	0.316	0.538
Batch 44)	1.037	0.728	0.438	0.320	0.531
Batch 45)	1.195	0.728	0.441	0.320	0.575
Batch 46)	0.841	0.793	0.470	0.363	0.610


...

Batch 33)	0.599	0.824	0.656	0.551	0.669
Batch 34)	0.457	0.874	0.595	0.521	0.725
Batch 35)	0.480	0.871	0.671	0.586	0.733
Batch 36)	0.590	0.834	0.623	0.522	0.684
Batch 37)	0.497	0.859	0.679	0.582	0.726
Batch 38)	0.437	0.882	0.659	0.558	0.730
Batch 39)	0.497	0.859	0.658	0.569	0.716
Batch 40)	0.478	0.858	0.648	0.560	0.721
Batch 41)	0.482	0.865	0.602	0.518	0.737
Batch 42)	0.397	0.892	0.646	0.558	0.729
Batch 43)	0.394	0.898	0.571	0.489	0.739
Batch 44)	0.566	0.844	0.552	0.456	0.694
Batch 45)	0.498	0.858	0.648	0.530	0.689
Batch 46)	0.558	0.844	0.688	0.593	0.705
Batch 47)	0.529	0.849	0.669	0.565	0.689
Batch 48)	0.476	0.857	0.611	0.499	0.689
Batch 49)	0.481	0.860	0.624	0.535	0.718
Batch 50)	0.430	0.881	0.641	0.538	0.736
Batch 51)	0.453	0.872	0.577	0.509	0.740
Batch 52)	0.616	0.820	0.568	0.479	0.689
Batch 53)	0.407	0.888	0.632	0.557	0.754
Batch 54)	0.554	0.833	0.590	0.494	0.703
Batch 55)	0.342	0.508	0.388	0.324	0.412
Train:	[0.5005199826664204, 0.8576208359058023, 0.6307815342014019, 0.5362903031

Batch 20)	0.388	0.894	0.692	0.610	0.772
Batch 21)	0.420	0.880	0.686	0.606	0.748
Batch 22)	0.429	0.878	0.629	0.541	0.752
Batch 23)	0.515	0.861	0.575	0.479	0.703
Batch 24)	0.543	0.852	0.653	0.561	0.720
Batch 25)	0.608	0.817	0.633	0.521	0.632
Batch 26)	0.403	0.879	0.642	0.550	0.761
Batch 27)	0.337	0.908	0.673	0.600	0.797
Batch 28)	0.460	0.866	0.618	0.535	0.743
Batch 29)	0.565	0.829	0.637	0.532	0.682
Batch 30)	0.454	0.877	0.677	0.583	0.748
Batch 31)	0.463	0.874	0.622	0.536	0.737
Batch 32)	0.480	0.866	0.651	0.547	0.681
Batch 33)	0.485	0.865	0.589	0.510	0.728
Batch 34)	0.508	0.857	0.621	0.531	0.707
Batch 35)	0.370	0.902	0.628	0.549	0.773
Batch 36)	0.620	0.821	0.631	0.508	0.634
Batch 37)	0.648	0.810	0.612	0.503	0.661
Batch 38)	0.490	0.859	0.629	0.537	0.700
Batch 39)	0.573	0.835	0.627	0.529	0.696
Batch 40)	0.444	0.880	0.669	0.580	0.741
Batch 41)	0.520	0.849	0.629	0.514	0.681
Batch 42)	0.415	0.877	0.654	0.575	0.761
Batch 43)	0.575	0.838	0.585	0.489	0.672
Batch 44)	0.379	0.895	0.641	0.569	0.778


Batch 7)	0.492	0.863	0.618	0.523	0.742
Batch 8)	0.393	0.889	0.694	0.603	0.758
Batch 9)	0.553	0.834	0.648	0.525	0.681
Batch 10)	0.457	0.870	0.627	0.534	0.728
Batch 11)	0.433	0.882	0.672	0.573	0.710
Batch 12)	0.349	0.898	0.692	0.623	0.787
Batch 13)	0.559	0.828	0.667	0.551	0.655
Batch 14)	0.547	0.845	0.618	0.505	0.694
Batch 15)	0.483	0.863	0.588	0.501	0.711
Batch 16)	0.465	0.859	0.677	0.550	0.668
Batch 17)	0.408	0.887	0.646	0.556	0.731
Batch 18)	0.588	0.832	0.650	0.548	0.657
Batch 19)	0.507	0.849	0.672	0.565	0.695
Batch 20)	0.407	0.885	0.730	0.624	0.746
Batch 21)	0.383	0.890	0.685	0.589	0.753
Batch 22)	0.399	0.884	0.616	0.539	0.768
Batch 23)	0.534	0.847	0.663	0.554	0.704
Batch 24)	0.452	0.862	0.615	0.516	0.723
Batch 25)	0.346	0.904	0.649	0.582	0.796
Batch 26)	0.393	0.888	0.683	0.606	0.766
Batch 27)	0.602	0.831	0.662	0.553	0.677
Batch 28)	0.480	0.874	0.649	0.571	0.746
Batch 29)	0.395	0.896	0.564	0.495	0.763
Batch 30)	0.369	0.889	0.627	0.548	0.764
Batch 31)	0.442	0.875	0.644	0.559	0.760
Bat

Val 999)	2.106	0.378	0.410	0.240	0.206
Val 1099)	0.115	0.991	0.500	0.491	0.965
Val metric:	[0.7092906773898459, 0.8072177439786029, 0.5519168429860404, 0.44865328530986576, 0.6464751485421634]

Epoch 68/199
----------
Batch 0)	0.386	0.889	0.574	0.509	0.771
Batch 1)	0.373	0.900	0.641	0.568	0.770
Batch 2)	0.621	0.814	0.671	0.555	0.660
Batch 3)	0.427	0.884	0.689	0.570	0.715
Batch 4)	0.338	0.902	0.665	0.590	0.785
Batch 5)	0.484	0.856	0.643	0.554	0.730
Batch 6)	0.448	0.883	0.628	0.547	0.749
Batch 7)	0.452	0.875	0.658	0.558	0.728
Batch 8)	0.480	0.861	0.661	0.581	0.723
Batch 9)	0.449	0.872	0.651	0.561	0.725
Batch 10)	0.448	0.879	0.663	0.542	0.702
Batch 11)	0.384	0.884	0.670	0.585	0.773
Batch 12)	0.395	0.891	0.641	0.569	0.760
Batch 13)	0.353	0.903	0.684	0.608	0.790
Batch 14)	0.459	0.872	0.644	0.559	0.728
Batch 15)	0.531	0.839	0.636	0.547	0.713
Batch 16)	0.484	0.860	0.669	0.574	0.735
Batch 17)	0.388	0.891	0.654	0.571	0.748
Batch 18)	0.506	0.855	0.637	0.546	0.722
Batch 19)	0.533	0.852	0.692	0.56

Batch 55)	0.291	0.513	0.421	0.356	0.453
Train:	[0.43150969600077155, 0.8766753359232878, 0.6660360644788753, 0.5740005654610557, 0.7374771181753211]
Val 99)	0.713	0.839	0.351	0.287	0.652
Val 199)	0.511	0.851	0.510	0.435	0.712
Val 299)	0.464	0.853	0.564	0.476	0.672
Val 399)	0.285	0.936	0.500	0.461	0.849
Val 499)	0.277	0.938	0.906	0.822	0.835
Val 599)	0.394	0.894	0.602	0.533	0.787
Val 699)	0.416	0.916	0.544	0.497	0.828
Val 799)	0.280	0.928	0.333	0.314	0.855
Val 899)	2.757	0.427	0.507	0.241	0.204
Val 999)	0.505	0.868	0.500	0.428	0.731
Val 1099)	0.889	0.718	0.506	0.370	0.404
Val metric:	[0.7070081658212051, 0.8076036799107895, 0.5507343806900523, 0.44776287392751857, 0.6458891686414706]

Epoch 71/199
----------
Batch 0)	0.510	0.852	0.693	0.589	0.720
Batch 1)	0.503	0.853	0.615	0.518	0.713
Batch 2)	0.453	0.867	0.658	0.563	0.734
Batch 3)	0.457	0.866	0.694	0.578	0.744
Batch 4)	0.463	0.870	0.682	0.585	0.720
Batch 5)	0.423	0.875	0.632	0.547	0.731
Batch 6)	0.389	0.891	0.650	0.575	0.760
Batch 7)	0

In [7]:
# continue after 60 epoch because of the power loss 

LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32_google = FCN32_google()
fcn32dict = LoadDict('fcn32_google_epoch59')
fcn32_google.load_state_dict(fcn32dict, strict = True)

fcn32_google_best, M = train_model(fcn32_google, device, train_loader, val_loader, lr = 5**(-5), dict_name = 'fcn32_google', num_epochs=140)

SaveDict(fcn32_google_best, 'fcn32_google_best')
SaveDict(M, 'fcn32_google_metric')

Epoch 0/139
----------
Batch 0)	0.400	0.889	0.613	0.531	0.746
Batch 1)	0.480	0.864	0.637	0.546	0.734
Batch 2)	0.564	0.837	0.649	0.558	0.681
Batch 3)	0.532	0.848	0.613	0.520	0.713
Batch 4)	0.460	0.873	0.612	0.532	0.738
Batch 5)	0.606	0.825	0.598	0.498	0.677
Batch 6)	0.559	0.836	0.682	0.577	0.694
Batch 7)	0.623	0.825	0.597	0.483	0.673
Batch 8)	0.552	0.834	0.611	0.491	0.682
Batch 9)	0.486	0.865	0.629	0.535	0.710
Batch 10)	0.375	0.897	0.681	0.598	0.772
Batch 11)	0.589	0.836	0.679	0.573	0.680
Batch 12)	0.560	0.842	0.593	0.488	0.644
Batch 13)	0.397	0.899	0.603	0.533	0.773
Batch 14)	0.491	0.860	0.599	0.492	0.703
Batch 15)	0.537	0.850	0.574	0.489	0.720
Batch 16)	0.621	0.829	0.575	0.474	0.697
Batch 17)	0.413	0.877	0.654	0.561	0.739
Batch 18)	0.589	0.829	0.610	0.505	0.679
Batch 19)	0.393	0.890	0.680	0.609	0.780
Batch 20)	0.498	0.855	0.625	0.545	0.724
Batch 21)	0.459	0.872	0.652	0.567	0.741
Batch 22)	0.489	0.859	0.646	0.541	0.696
Batch 23)	0.454	0.872	0.607	0.527	0.735
Batch 24)	0.428	0.882	0.726

Val 199)	0.969	0.766	0.661	0.534	0.584
Val 299)	1.724	0.575	0.573	0.444	0.431
Val 399)	0.243	0.952	0.250	0.227	0.822
Val 499)	0.948	0.757	0.495	0.366	0.432
Val 599)	0.492	0.880	0.362	0.305	0.662
Val 699)	0.553	0.856	0.728	0.633	0.690
Val 799)	0.253	0.967	0.333	0.310	0.866
Val 899)	0.564	0.857	0.565	0.469	0.659
Val 999)	0.921	0.779	0.598	0.472	0.583
Val 1099)	0.365	0.904	0.520	0.462	0.779
Val metric:	[0.7243010660819542, 0.8034725696414701, 0.5393868441572987, 0.43567261036819327, 0.6379901768389138]

Epoch 3/139
----------
Batch 0)	0.425	0.885	0.656	0.565	0.724
Batch 1)	0.456	0.869	0.632	0.547	0.715
Batch 2)	0.476	0.865	0.643	0.531	0.741
Batch 3)	0.421	0.885	0.616	0.545	0.746
Batch 4)	0.441	0.872	0.688	0.607	0.748
Batch 5)	0.463	0.872	0.636	0.523	0.717
Batch 6)	0.375	0.896	0.657	0.573	0.773
Batch 7)	0.440	0.871	0.663	0.575	0.720
Batch 8)	0.517	0.857	0.644	0.547	0.719
Batch 9)	0.608	0.841	0.606	0.505	0.697
Batch 10)	0.597	0.835	0.630	0.525	0.663
Batch 11)	0.475	0.860	0.631	0.540	0.721
B

Batch 47)	0.471	0.867	0.655	0.540	0.678
Batch 48)	0.431	0.873	0.654	0.570	0.759
Batch 49)	0.412	0.886	0.711	0.620	0.742
Batch 50)	0.461	0.870	0.675	0.576	0.741
Batch 51)	0.394	0.900	0.672	0.586	0.772
Batch 52)	0.376	0.901	0.607	0.536	0.765
Batch 53)	0.398	0.877	0.670	0.591	0.767
Batch 54)	0.427	0.879	0.648	0.569	0.748
Batch 55)	0.243	0.530	0.378	0.324	0.448
Train:	[0.4620129899155322, 0.8682054869765187, 0.6501337138707346, 0.5574184256307866, 0.7268325221776298]
Val 99)	1.108	0.751	0.333	0.234	0.493
Val 199)	0.299	0.900	0.379	0.317	0.683
Val 299)	0.334	0.902	0.541	0.477	0.763
Val 399)	0.483	0.908	0.525	0.477	0.818
Val 499)	0.069	0.993	0.500	0.494	0.976
Val 599)	1.148	0.609	0.598	0.373	0.355
Val 699)	0.530	0.822	0.527	0.421	0.624
Val 799)	0.747	0.810	0.677	0.591	0.674
Val 899)	0.459	0.859	0.603	0.519	0.703
Val 999)	0.643	0.850	0.251	0.202	0.647
Val 1099)	1.449	0.548	0.431	0.239	0.309
Val metric:	[0.7206795700757851, 0.8043609413554793, 0.5428897191538334, 0.43937554031411724, 0.6402634

Batch 34)	0.408	0.886	0.667	0.583	0.749
Batch 35)	0.380	0.893	0.680	0.602	0.747
Batch 36)	0.451	0.863	0.656	0.560	0.710
Batch 37)	0.444	0.873	0.661	0.572	0.754
Batch 38)	0.379	0.898	0.730	0.621	0.724
Batch 39)	0.497	0.863	0.660	0.543	0.710
Batch 40)	0.312	0.920	0.714	0.626	0.804
Batch 41)	0.434	0.876	0.648	0.559	0.744
Batch 42)	0.423	0.875	0.649	0.572	0.745
Batch 43)	0.402	0.894	0.652	0.570	0.734
Batch 44)	0.593	0.826	0.650	0.541	0.688
Batch 45)	0.330	0.908	0.678	0.586	0.760
Batch 46)	0.484	0.852	0.623	0.531	0.685
Batch 47)	0.416	0.876	0.645	0.564	0.745
Batch 48)	0.496	0.856	0.648	0.557	0.720
Batch 49)	0.558	0.838	0.664	0.548	0.694
Batch 50)	0.350	0.902	0.680	0.580	0.733
Batch 51)	0.387	0.889	0.637	0.562	0.765
Batch 52)	0.432	0.874	0.629	0.544	0.737
Batch 53)	0.376	0.897	0.686	0.613	0.780
Batch 54)	0.474	0.861	0.628	0.527	0.710
Batch 55)	0.270	0.524	0.388	0.325	0.425
Train:	[0.44370823573294305, 0.8732985535069531, 0.6606366027173841, 0.5680033199161778, 0.7331210788914261]
Val 99)	0.6

Batch 21)	0.540	0.836	0.657	0.549	0.707
Batch 22)	0.480	0.866	0.706	0.619	0.724
Batch 23)	0.468	0.876	0.703	0.598	0.719
Batch 24)	0.382	0.899	0.664	0.554	0.738
Batch 25)	0.432	0.873	0.663	0.547	0.701
Batch 26)	0.438	0.879	0.625	0.535	0.746
Batch 27)	0.316	0.905	0.692	0.610	0.773
Batch 28)	0.392	0.879	0.676	0.582	0.764
Batch 29)	0.331	0.913	0.767	0.668	0.739
Batch 30)	0.427	0.877	0.672	0.594	0.755
Batch 31)	0.487	0.856	0.684	0.572	0.705
Batch 32)	0.510	0.854	0.737	0.593	0.681
Batch 33)	0.323	0.913	0.692	0.613	0.790
Batch 34)	0.471	0.860	0.702	0.610	0.719
Batch 35)	0.330	0.909	0.602	0.542	0.791
Batch 36)	0.453	0.871	0.625	0.539	0.722
Batch 37)	0.614	0.818	0.679	0.560	0.680
Batch 38)	0.341	0.905	0.710	0.637	0.782
Batch 39)	0.426	0.877	0.597	0.513	0.735
Batch 40)	0.410	0.880	0.674	0.581	0.714
Batch 41)	0.527	0.837	0.641	0.508	0.690
Batch 42)	0.439	0.871	0.683	0.598	0.755
Batch 43)	0.419	0.879	0.637	0.558	0.719
Batch 44)	0.457	0.870	0.686	0.591	0.732
Batch 45)	0.531	0.846	0.643	0.549	0.737


...

Batch 0)	0.133	0.960	0.835	0.748	0.865
Batch 1)	0.128	0.964	0.845	0.765	0.875
Batch 2)	0.120	0.965	0.838	0.760	0.876
Batch 3)	0.130	0.965	0.860	0.744	0.837
Batch 4)	0.108	0.970	0.821	0.740	0.881
Batch 5)	0.131	0.965	0.828	0.746	0.864
Batch 6)	0.140	0.960	0.881	0.794	0.857
Batch 7)	0.120	0.965	0.858	0.747	0.839
Batch 8)	0.154	0.958	0.843	0.725	0.804
Batch 9)	0.129	0.962	0.874	0.760	0.807
Batch 10)	0.117	0.967	0.900	0.785	0.833
Batch 11)	0.151	0.952	0.867	0.773	0.840
Batch 12)	0.145	0.956	0.829	0.730	0.845
Batch 13)	0.129	0.963	0.783	0.677	0.834
Batch 14)	0.130	0.962	0.837	0.751	0.851
Batch 15)	0.158	0.954	0.851	0.777	0.846
Batch 16)	0.108	0.969	0.841	0.765	0.879
Batch 17)	0.164	0.952	0.879	0.798	0.842
Batch 18)	0.159	0.951	0.865	0.752	0.798
Batch 19)	0.128	0.965	0.867	0.773	0.868
Batch 20)	0.142	0.957	0.784	0.709	0.859
Batch 21)	0.137	0.960	0.900	0.815	0.862
Batch 22)	0.124	0.963	0.830	0.741	0.848
Batch 23)	0.162	0.955	0.819	0.720	0.790
Batch 24)	0.109	0.969	0.877	0.802	0.862
Batch 25)	

Val 299)	1.079	0.753	0.505	0.361	0.506
Val 399)	0.597	0.873	0.761	0.660	0.751
Val 499)	0.669	0.861	0.739	0.659	0.726
Val 599)	0.032	0.992	0.500	0.492	0.970
Val 699)	2.665	0.449	0.428	0.191	0.266
Val 799)	0.778	0.858	0.648	0.575	0.728
Val 899)	1.152	0.694	0.622	0.452	0.477
Val 999)	0.676	0.810	0.742	0.633	0.666
Val 1099)	0.286	0.942	0.199	0.175	0.768
Val metric:	[0.850053303513733, 0.8011969017244611, 0.5546963986598119, 0.4510516550461195, 0.6438217250322198]

Epoch 133/139
----------
Batch 0)	0.100	0.973	0.817	0.749	0.898
Batch 1)	0.091	0.976	0.832	0.763	0.883
Batch 2)	0.173	0.953	0.859	0.723	0.747
Batch 3)	0.152	0.957	0.828	0.739	0.846
Batch 4)	0.185	0.949	0.784	0.698	0.824
Batch 5)	0.104	0.971	0.894	0.809	0.880
Batch 6)	0.140	0.960	0.828	0.742	0.858
Batch 7)	0.147	0.955	0.802	0.722	0.843
Batch 8)	0.108	0.970	0.850	0.763	0.885
Batch 9)	0.122	0.965	0.846	0.765	0.879
Batch 10)	0.120	0.963	0.869	0.784	0.863
Batch 11)	0.153	0.953	0.828	0.735	0.821
Batch 12)	0.157	0.954	0.850	0.730	0.792


Batch 48)	0.160	0.955	0.893	0.787	0.828
Batch 49)	0.142	0.958	0.824	0.735	0.854
Batch 50)	0.126	0.962	0.817	0.726	0.845
Batch 51)	0.121	0.965	0.834	0.746	0.865
Batch 52)	0.174	0.951	0.867	0.766	0.833
Batch 53)	0.098	0.973	0.806	0.723	0.859
Batch 54)	0.132	0.963	0.859	0.780	0.862
Batch 55)	0.123	0.563	0.490	0.434	0.480
Train:	[0.13368000912902167, 0.9612648522695642, 0.8491725884276287, 0.7582613485457766, 0.8466544470661529]
Val 99)	0.787	0.793	0.459	0.380	0.590
Val 199)	0.516	0.838	0.756	0.595	0.531
Val 299)	0.456	0.878	0.883	0.751	0.701
Val 399)	0.748	0.809	0.498	0.423	0.650
Val 499)	0.326	0.889	0.677	0.607	0.754
Val 599)	0.314	0.931	0.499	0.443	0.779
Val 699)	0.265	0.920	0.500	0.453	0.818
Val 799)	0.613	0.847	0.500	0.408	0.662
Val 899)	1.300	0.860	0.500	0.422	0.711
Val 999)	1.356	0.673	0.623	0.434	0.446
Val 1099)	1.452	0.658	0.383	0.263	0.427
Val metric:	[0.869274262783162, 0.7971949994866382, 0.5382045826622394, 0.4346215252788924, 0.6339186404449919]

Epoch 136/139
----------
Batc

Batch 35)	0.109	0.968	0.843	0.738	0.847
Batch 36)	0.107	0.967	0.846	0.742	0.838
Batch 37)	0.132	0.964	0.885	0.803	0.853
Batch 38)	0.129	0.963	0.769	0.691	0.864
Batch 39)	0.166	0.950	0.774	0.691	0.837
Batch 40)	0.153	0.955	0.825	0.735	0.850
Batch 41)	0.146	0.956	0.796	0.673	0.793
Batch 42)	0.100	0.973	0.864	0.769	0.856
Batch 43)	0.138	0.962	0.880	0.799	0.858
Batch 44)	0.168	0.952	0.815	0.719	0.836
Batch 45)	0.123	0.964	0.864	0.759	0.837
Batch 46)	0.145	0.956	0.875	0.783	0.844
Batch 47)	0.105	0.968	0.861	0.758	0.839
Batch 48)	0.124	0.962	0.843	0.760	0.862
Batch 49)	0.105	0.968	0.874	0.797	0.876
Batch 50)	0.097	0.972	0.907	0.826	0.879
Batch 51)	0.109	0.968	0.905	0.816	0.840
Batch 52)	0.142	0.959	0.859	0.763	0.814
Batch 53)	0.106	0.970	0.829	0.761	0.893
Batch 54)	0.126	0.963	0.801	0.687	0.812
Batch 55)	0.130	0.562	0.500	0.440	0.487
Train:	[0.13193273689035032, 0.961778462180204, 0.8498379064640259, 0.7590949098736224, 0.8472042423503867]
Val 99)	0.132	0.965	0.768	0.729	0.917
Val 199)	1.916

0